<a href="https://colab.research.google.com/github/odibo1/BGP-HE-Web-Scraping/blob/master/Simplified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import necessary libraries
import pandas as pd
import numpy as np
import altair as alt

In [3]:
#function to read in all export files

def Read_Files():

    excel_files = ['Oceania.xlsx','LatAm (1).xlsx','Europe (T-U).xlsx','Europe (I-S).xlsx','Europe (A-H).xlsx','Asia_all.xlsx','Asia (Middle East).xlsx','Africa (rest excl Zim).xlsx','Africa (ZA&Kenya).xlsx','US (fiber).xlsx','US (all other).xlsx','Canada.xlsx', 'TeleGeography Business Broadband All Carrier Postings.xlsx' ]
    read_files_list = []
    error = 1

    for excel_file in excel_files:
        try:
            df = pd.read_excel(excel_file)
            read_files_list.append(df)
            error += 1
        except:
            print('Please check if file is downloaded, path name is accurate, or csv is saved in non-utf-8 format). File ',
                  str(error), 'did not read') #tells you which file had a problem reading in            
            break 
            
    return read_files_list

raw_files = Read_Files()
raw_files

[      Region    Country  ...              Created At              Updated At
 0    Oceania  Australia  ... 2020-05-13 18:40:31.390 2020-06-29 14:40:25.125
 1    Oceania  Australia  ... 2020-05-13 18:40:31.406 2020-05-13 18:40:31.406
 2    Oceania  Australia  ... 2020-05-13 18:40:31.437 2020-05-13 18:40:31.437
 3    Oceania  Australia  ... 2020-05-13 18:40:31.421 2020-06-29 14:40:37.706
 4    Oceania  Australia  ... 2020-06-29 15:49:55.432 2020-06-29 15:49:55.432
 ..       ...        ...  ...                     ...                     ...
 196  Oceania      Samoa  ... 2020-07-02 20:44:07.768 2020-07-06 22:48:29.139
 197  Oceania      Samoa  ... 2020-07-06 22:49:00.254 2020-07-06 22:49:00.254
 198  Oceania      Samoa  ... 2020-07-06 22:17:24.775 2020-07-06 22:17:46.702
 199  Oceania      Samoa  ... 2020-07-06 22:18:33.085 2020-07-06 22:19:00.873
 200  Oceania      Samoa  ... 2020-07-02 20:38:58.292 2020-07-06 22:17:04.770
 
 [201 rows x 29 columns],
                         Region  ...

In [4]:
#makes column name in all files the same
def Rename_Columns(raw_files):

    new_df_list = []

    for dataframe in raw_files:
        dataframe = dataframe.rename(columns = {'Date':'Quarter', 'Mbps (Upload)':'Mbps Up','Mbps (Download)':'Mbps Down', 'Tariff Name':'Tariff', 'Access':'Access Type',
                                    'Unlimited Data Allowance':'Unlimited Data', 'One-Time Fee (Local)':'NRC Local', 'MRC (Local)':'MRC Local',
                                    'One-Time Fee (USD)':'NRC USD', 'MRC (USD)':'MRC USD','Local Currency':'Currency'})
        new_df_list.append(dataframe)
    
    return new_df_list

raw_list2 = Rename_Columns(raw_files)
raw_list2

[      Region    Country  ...              Created At              Updated At
 0    Oceania  Australia  ... 2020-05-13 18:40:31.390 2020-06-29 14:40:25.125
 1    Oceania  Australia  ... 2020-05-13 18:40:31.406 2020-05-13 18:40:31.406
 2    Oceania  Australia  ... 2020-05-13 18:40:31.437 2020-05-13 18:40:31.437
 3    Oceania  Australia  ... 2020-05-13 18:40:31.421 2020-06-29 14:40:37.706
 4    Oceania  Australia  ... 2020-06-29 15:49:55.432 2020-06-29 15:49:55.432
 ..       ...        ...  ...                     ...                     ...
 196  Oceania      Samoa  ... 2020-07-02 20:44:07.768 2020-07-06 22:48:29.139
 197  Oceania      Samoa  ... 2020-07-06 22:49:00.254 2020-07-06 22:49:00.254
 198  Oceania      Samoa  ... 2020-07-06 22:17:24.775 2020-07-06 22:17:46.702
 199  Oceania      Samoa  ... 2020-07-06 22:18:33.085 2020-07-06 22:19:00.873
 200  Oceania      Samoa  ... 2020-07-02 20:38:58.292 2020-07-06 22:17:04.770
 
 [201 rows x 29 columns],
                         Region  ...

In [8]:
#combines all files into one
def Concat_Files(raw_list2):

    final_df = pd.concat(raw_list2, axis=0, ignore_index=True)

    return final_df

combine_df = Concat_Files(raw_list2)
combine_df

,Region,Country,Company,Tariff,Tariff URL,Access Type,Quarter,Mbps Up,Mbps Down,GB Allowance,GB Allowance Notes,Unlimited Data,NRC USD,MRC USD,NRC Local,MRC Local,Currency,Exchange Rate,Technologies,State(s),Geographic Notes,Notes,Static IP Notes,Static IP,24x7 Support,Router Included,Contract Length,Created At,Updated At,Geography Notes,Data Allowance (Gb),General Notes
0,Oceania,Australia,iiNet,Business Fibre 1000,https://assets.iinet.net.au/website/about/lega...,LAN/FTTx,Q2 2020,1000.000,1000.000,NaN,NaN,1.0,780.607936,624.912910,1098.0,879.00,AUD,0.710936,FTTP – Fiber-to-the-Premises,NaN,NaN,NaN,NaN,1.0,1.0,0.0,36.0,2020-05-13 18:40:31.390,2020-06-29 14:40:25.125,NaN,NaN,NaN
1,Oceania,Australia,TPG,Fibre 1000,http://www.tpg.com.au/business-products/intern...,LAN/FTTx,Q2 2020,1000.000,1000.000,NaN,NaN,1.0,781.318872,568.038015,1099.0,799.00,AUD,0.710936,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,36.0,2020-05-13 18:40:31.406,2020-05-13 18:40:31.406,NaN,NaN,NaN
2,Oceania,Australia,TPG,Fibre 400,http://www.tpg.com.au/business-products/intern...,LAN/FTTx,Q2 2020,400.000,400.000,NaN,NaN,1.0,1563.348680,284.374476,2199.0,400.00,AUD,0.710936,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,36.0,2020-05-13 18:40:31.437,2020-05-13 18:40:31.437,NaN,NaN,NaN
3,Oceania,Australia,iiNet,Business Fibre 400,https://assets.iinet.net.au/website/about/lega...,LAN/FTTx,Q2 2020,400.000,400.000,NaN,NaN,1.0,1563.348680,312.100987,2199.0,439.00,AUD,0.710936,FTTP – Fiber-to-the-Premises,NaN,NaN,NaN,NaN,1.0,1.0,0.0,36.0,2020-05-13 18:40:31.421,2020-06-29 14:40:37.706,NaN,NaN,NaN
4,Oceania,Australia,SingTel Optus,5G,https://www.optus.com.au/business/broadband-in...,Wireless,Q2 2020,NaN,138.000,NaN,NaN,1.0,142.187238,49.765533,200.0,70.00,AUD,0.710936,5G,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,2020-06-29 15:49:55.432,2020-06-29 15:49:55.432,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37924,US & Canada,United States,"Internet 2000, Inc. (I2K Wireless)",Business LMS3000 System 384KB/384KB,http://www.i2kwireless.com/pricing.htm,Wireless,Q2 2016,0.384,0.384,NaN,NaN,NaN,99.000000,60.000000,99.0,60.00,USD,NaN,WiMAX,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaT,NaT,"Michigan: Bangor, Benton Harbor, Cheboygan, Co...",NaN,1-year service agreement
37925,US & Canada,United States,Shelby Broadband,Basic Wireless (Business Plan),http://www.shelbybb.com/business_wireless.php,Wireless,Q2 2016,0.384,0.384,NaN,NaN,NaN,0.000000,50.000000,0.0,50.00,USD,NaN,WiMAX,NaN,NaN,NaN,1 available upon request,1.0,NaN,NaN,NaN,NaT,NaT,"Kentucky: Lexington, Georgetown, Paris, Frankf...",NaN,NaN
37926,US & Canada,United States,"Grand Valley Telecommunications, Inc.",256K High Speed Internet Basic (Business),Unknown,DSL,Q2 2016,0.256,0.256,NaN,NaN,NaN,25.000000,18.000000,25.0,18.00,USD,NaN,ADSL,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaT,NaT,Colorado: Mesa and Montrose counties,NaN,GVTCI will rent a modem for only $3 mo with a ...
37927,US & Canada,United States,"Hopi Telecommunications, Inc.",Plan 1,http://www.hopitelecom.com/services-dsl.php,DSL,Q2 2016,NaN,0.256,NaN,NaN,NaN,240.000000,29.950000,240.0,29.95,USD,NaN,NaN,NaN,NaN,NaN,Static IP Address: $5/month ($10 per additiona...,1.0,NaN,NaN,NaN,NaT,NaT,"Arizona: Hotevilla-Bacavi, Oraibi, Kykotsmovi ...",NaN,Static IP Address: $5/month ($10 per additiona...


In [9]:
#include sub region information from subregion file 
sub_regions = pd.read_excel('Country Regions Subregions.xlsx')
sub_regions = sub_regions.drop(columns=['country_id','subregion_id','Region','region_id'])

df_merge = combine_df.merge(sub_regions, how='left', on='Country')
df_merge.columns #check to see if column has been added

Index(['Region', 'Country', 'Company', 'Tariff', 'Tariff URL', 'Access Type',
       'Quarter', 'Mbps Up', 'Mbps Down', 'GB Allowance', 'GB Allowance Notes',
       'Unlimited Data', 'NRC USD', 'MRC USD', 'NRC Local', 'MRC Local',
       'Currency', 'Exchange Rate', 'Technologies', 'State(s)',
       'Geographic Notes', 'Notes', 'Static IP Notes', 'Static IP',
       '24x7 Support', 'Router Included', 'Contract Length', 'Created At',
       'Updated At', 'Geography Notes', 'Data Allowance (Gb)', 'General Notes',
       'subregion_name'],
      dtype='object')

In [10]:
#create speed category column for speed buckets and boolean column for speed greater/less than 50

def Speed_Categories(df_merge):
    
    speed_dict = {0:'0-10 mbps', 11:'11-50mbps', 51:'51-100mbps', 101:'101-500mbps', 501:'501-999mbps', 1000:'>=1000mbps'}
    
    for i in df_merge.index:

        for key in speed_dict:

              if df_merge.at[i, 'Mbps Down'] >= key:
                  df_merge.at[i, 'Mbps Down (categories)'] = speed_dict[key]
              
    for i in df_merge.index:

        if df_merge.at[i, 'Mbps Down'] > 50:
            df_merge.at[i, '<50mbps'] = False
        elif df_merge.at[i, 'Mbps Down'] <= 50:
            df_merge.at[i, '<50mbps'] = True
    
    return df_merge

final_df = Speed_Categories(df_merge)
final_df[['Mbps Down','Mbps Down (categories)','<50mbps']].sample(10)

,Mbps Down,Mbps Down (categories),<50mbps
2933,50.0,11-50mbps,True
32480,100.0,51-100mbps,False
7554,60.0,51-100mbps,False
32872,110.0,101-500mbps,False
13189,20.0,11-50mbps,True
1291,3.0,0-10 mbps,True
3257,40.0,11-50mbps,True
14688,3.0,0-10 mbps,True
20777,1000.0,>=1000mbps,False
34969,15.0,11-50mbps,True


In [11]:
#check if upload speeds are greater than download speeds
speed_errors = final_df[final_df['Mbps Down'] < final_df ['Mbps Up']]

speed_errors[['Company','Tariff','Mbps Down', 'Mbps Up', 'Quarter']][speed_errors['Quarter']=='Q2 2020']


,Company,Tariff,Mbps Down,Mbps Up,Quarter
1615,StarNet,Starnet 60 Mbps,60.0,150.0,Q2 2020
1618,StarNet,Starnet 40 Mbps,40.0,100.0,Q2 2020
1624,StarNet,Starnet 20 Mbps,20.0,50.0,Q2 2020
4208,TS2,E70B 512kbps,1.0,2.0,Q2 2020
4676,TS2,EUTELSAT 70B Wide 1/2,1.0,2.0,Q2 2020
5639,Afrihost (incl. Axxess DSL),Vuma Fibre 1000/100,100.0,1000.0,Q2 2020
5721,Afrihost (incl. Axxess DSL),Evotel Fibre 1000/100,100.0,1000.0,Q2 2020
5878,Cape Connect,Octotel Fibre 1000/25,25.0,1000.0,Q2 2020
6061,Cool Ideas,Openserve Super Cool 20/10,10.0,20.0,Q2 2020
6190,MWEB (South Africa),Fiber 10Mbps Unlimited Frogfoot,10.0,15.0,Q2 2020


In [12]:
#graph showing the composition of different Access Types in the market over time
##specify whether region or country and what region or country

def AccessType_Stacked_Graph(Region_or_Country, RegionName_or_CountryName, final_df):

    document = final_df[final_df[Region_or_Country]==RegionName_or_CountryName] #New dataframe of only country/region plans (document = df_merge for worldwide plans)

    document = document[(document['Quarter']=='Q2 2017') | (document['Quarter']=='Q2 2020') ] #filter years to include only 2017 and 2020
    document = document[document['Mbps Down (categories)']=='11-50mbps'] #filter speeds to only include 11-50mbps speeds
    
    domain = ['Cable','DSL','LAN/FTTx','Other','Satellite','Wireless','Ethernet'] 
    range =  ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#17becf', '#e377c2'] #assign colors to access types
    
    stacked = alt.Chart(document).mark_bar().encode(
        x=alt.X('count(Access Type)', stack="normalize",title='Percentage of Access Type'),
        y='Quarter',
        color=alt.Color('Access Type', scale=alt.Scale(domain=domain, range=range))
    ).properties(title=RegionName_or_CountryName
    )

    return stacked

AccessType_Stacked_Graph('Region', 'Latin America & Caribbean', final_df)

alt.Chart(...)

In [13]:
#graph showing the percentage composition of different Access Types in the market over time
##specify whether region or country and what region or country

def Quarter_Percentages(Region_or_Country, RegionName_or_CountryName, final_df):

    document = final_df[final_df[Region_or_Country]==RegionName_or_CountryName]

    quarter_list = document['Quarter'].unique()

    df_list = []

    #add each line of value counts as a dataframe to list and concat
    for year in quarter_list:

        Quarter = document[document['Quarter']==year]
        
        df = round(Quarter['Access Type'].value_counts(normalize=True),2)
        df_value_count = pd.DataFrame(df.reset_index()) 
        df_value_count.columns = ['Access Type', year]
        df_value_count = df_value_count.set_index('Access Type')
        df_list.append(df_value_count)

    percent_df = pd.concat(df_list, axis = 1)
    percent_df = percent_df.rename_axis(index=RegionName_or_CountryName)
    return percent_df

Quarter_Percentages('Region', 'Latin America & Caribbean', final_df)

,Q2 2020,Q2 2019,Q2 2018,Q2 2017,Q2 2016
Latin America & Caribbean,,,,,
LAN/FTTx,0.56,0.46,0.33,0.21,0.19
Wireless,0.18,0.29,0.28,0.32,0.32
Cable,0.15,0.13,0.09,0.11,0.14
DSL,0.09,0.09,0.16,0.24,0.26
Other,0.02,0.03,0.14,0.12,0.08
Satellite,0.00,NaN,NaN,NaN,NaN


In [14]:
#graph showing the count composition of different Access Types in the market over time
##specify whether region or country and what region or country

def Quarter_Count(Region_or_Country, RegionName_or_CountryName, final_df):

    document = final_df[final_df[Region_or_Country]==RegionName_or_CountryName]

    quarter_list = document['Quarter'].unique()

    df_list = []

    for year in quarter_list:

        Quarter = document[document['Quarter']==year]
        
        df = Quarter['Access Type'].value_counts()
        df_value_count = pd.DataFrame(df.reset_index())
        df_value_count.columns = ['Access Type', year]
        df_value_count = df_value_count.set_index('Access Type')
        df_list.append(df_value_count)

    percent_df = pd.concat(df_list, axis = 1)

    return percent_df

Quarter_Count('Region', 'Oceania', final_df)

,Q2 2020,Q2 2019,Q2 2018,Q2 2017,Q2 2016
LAN/FTTx,99.0,92.0,123,103,108.0
DSL,47.0,54.0,101,75,87.0
Wireless,43.0,38.0,43,45,45.0
Cable,12.0,19.0,5,4,7.0
Other,NaN,NaN,5,4,NaN


In [16]:
#get number of fibre providers/plans in each country in a region

def Fiber_Plan_Provider_Count(Region, RegionName, Company_or_Tariff, final_df):

    document = final_df[final_df[Region]==RegionName]

    document = document[document['Access Type']=='LAN/FTTx']

    quarter_list = document['Quarter'].unique()

    new_dict = {}
    
    country_list = document['Country'].unique()

    df_list=[]

    #for each country, add a dictionary of number of plans/providers by year
    #turn each dictionary to a dataframe, add each dateframe to list and concat from list 
    for country in country_list:
        document2 = document[document['Country']==country]
        for i in quarter_list:

            document1 = document2[document2['Quarter']==i]
            new_dict[i] = len(document1[Company_or_Tariff].unique())

        df = pd.DataFrame(new_dict, index = [country] )
        df_list.append(df)

    final = pd.concat(df_list)
    final = final.reset_index().rename(columns={'index':'Country'})
 
    return final

fibre = Fiber_Plan_Provider_Count('Region', 'Africa', 'Tariff', df_merge)

fibre_df = fibre.sort_values(by='Q2 2020', ascending=False).head(10) #filters datframe to 10 highest plans/providers in 2020 (remove .head(10) for all countries)
fibre_df

,Country,Q2 2020,Q2 2019,Q2 2018,Q2 2017,Q2 2016
28,South Africa,524,216,78,44,48
27,Kenya,26,26,19,15,14
21,Swaziland,26,0,0,0,0
14,Namibia,21,21,20,4,0
6,Equatorial Guinea,20,20,3,0,0
16,Nigeria,17,18,19,19,23
10,Malawi,17,17,17,0,0
2,Botswana,11,11,7,0,0
4,Cape Verde,9,4,0,0,0
25,Uganda,9,10,3,4,0


In [17]:
#print chart of fibre plans/providers in top 10 countries

def Fibre_Plan_Provider_Graph(fibre_df,Plans_or_Providers):

    long_fibre = pd.melt(fibre_df, id_vars = 'Country', var_name = 'Quarter', value_name = 'Fibre Providers') #changes dataframe to long format

    long_fibre = long_fibre[(long_fibre['Quarter']=='Q2 2020') | (long_fibre['Quarter']=='Q2 2017')]

    fibre_graph = alt.Chart(long_fibre).mark_bar().encode(
        x=alt.X('Quarter:O',title=None),
        y=alt.Y('Fibre Providers:Q', axis= alt.Axis(title=Plans_or_Providers)), #change MRC Local to NRC Local or any of the other price columns to see trends in that price
        color= alt.Color('Quarter:N', legend=None),
        column=alt.Column('Country:N',title=None)
    )

    return fibre_graph

Fibre_Plan_Provider_Graph(fibre_df, 'Plans')

alt.Chart(...)

In [ ]:
#export dataframes as excel files
def Excel_Export(fibre_df, File_Name_xlsx):

    return files2.to_excel(File_Name_xlsx)


In [25]:
def Median_Access_Type(Region_or_Country, RegionName_or_CountryName, final_df):

    document = final_df[final_df[Region_or_Country]==RegionName_or_CountryName]

    #include (or exclude by changing == to != and | to &) companies/Access Type/Quarter/Speed
    #document = document[(document['Company']=='Rogers Communication') | (document['Company']=='Shaw Business') | (document['Company']=='Bell Aliant') | (document['Company']=='Telus') ]
    document = document[(document['Access Type']!='Other') & (document['Access Type']!='Satellite')]
    document = document[(document['Quarter']=='Q2 2017') | (document['Quarter']=='Q2 2020')]
    document = document[document['Mbps Down (categories)']=='11-50mbps']

    graph = alt.Chart(document).mark_bar().encode(
    x='Quarter:O',
    y='median(MRC USD):Q', #change MRC Local to NRC Local or any of the other price columns to see trends in that price
    color= alt.Color('Quarter:N', legend=None),
    column='Access Type:N'
    ).properties(title = RegionName_or_CountryName )
    

    return graph

Median_Access_Type('Region', 'Africa', final_df)

alt.Chart(...)

In [30]:
def CAGR_Calculator(Region_or_Country, RegionName_or_CountryName, final_df):
  
    CAGR = final_df[final_df[Region_or_Country]==RegionName_or_CountryName] #remove for regional CAGR

    CAGR_df = CAGR[['Country','MRC USD','Mbps Down','Quarter','Access Type']]
    CAGR_df = CAGR_df[CAGR_df['Access Type']=='Wireless']
    CAGR_df = CAGR_df[CAGR_df['Mbps Down'] < 50 ]

    #get dataframe of cross-year medians
    temp4 = round(CAGR_df.groupby(['Quarter','Country'])['MRC USD'].median(),4) #change Country to Region for regional CAGR
    temp5 = pd.DataFrame(temp4).reset_index()
    temp5
    
    #create and calculate CAGR column
    source1 = temp5.pivot(index='Country', columns='Quarter', values='MRC USD').reset_index() #change Country to Region for regional CAGR
    source1['CAGR'] = round((((source1['Q2 2020']/source1['Q2 2017'])**(1/3))-1), 4) #CAGR calcultion
    source2 = source1.sort_values(by='CAGR', ascending=False)
   
    return source2

CAGR_file = CAGR_Calculator('Region', 'Africa', final_df)
CAGR_file

Quarter,Country,Q2 2016,Q2 2017,Q2 2018,Q2 2019,Q2 2020,CAGR
33,Sierra Leone,48.4696,48.4696,51.0598,440.0000,520.0000,1.2055
26,Mauritius,26.0340,26.0340,112.4784,142.6555,162.3067,0.8405
12,Côte d'Ivoire,64.2841,64.2841,60.5185,66.8018,258.7353,0.5907
31,Nigeria,40.7703,29.0226,47.1899,55.4912,56.6798,0.2500
4,Burkina Faso,527.3237,414.4517,418.2899,19.6980,736.5332,0.2113
19,Lesotho,71.2581,71.2581,95.4391,121.5332,121.0674,0.1932
1,Angola,52.5383,52.5383,82.5693,72.1285,82.4706,0.1622
21,Libya,21.7302,21.7302,36.7012,35.8320,30.5861,0.1207
18,Kenya,49.3791,47.0000,47.0000,64.4177,64.9501,0.1139
15,Ethiopia,124.9413,124.9521,125.8108,189.1342,161.2153,0.0886


In [39]:
def CAGR_Only(CAGR_file):

    return alt.Chart(CAGR_file).mark_bar().encode(
        x=alt.X('Country', title=None), #change country to region for regional chart
        y=alt.Y('CAGR', axis=alt.Axis(title='2017-2020 CAGR', format='%'))
    ).properties(title='CAGR <50 LatAm',width=300)

CAGR_Only(CAGR_file)
    

alt.Chart(...)

In [38]:
def CAGR_Graph(CAGR_file): #use dataframe from CAGR calculation

    base = alt.Chart(CAGR_file).encode(
        alt.X('Country', axis=alt.Axis(title=None)) #change Country to Region for Region chart
    )

    bar = base.mark_bar(color='#57A44C').encode(
        y=alt.Y('Q2 2020', axis=alt.Axis(title='Median MRC ($)', format='$.2f', titleColor='#57A44C'))
    ).properties(
        width=alt.Step(40)  # controls width of bar.
    ).properties(width=400,height=300)

    tick = base.mark_tick(
        color='red',
        thickness=2,
        size=10 * 0.9,  # controls width of tick.
    
    ).encode(
        y=alt.Y('CAGR', axis=alt.Axis(title='2017-2020 CAGR', titleColor='red',format='%'))
    ).properties(width=500,height=300)

    return alt.layer(bar, tick).resolve_scale(
            y = 'independent'
            ).properties(width=500,height=300)

CAGR_Graph(CAGR_file)

alt.LayerChart(...)